In [1]:
# Importação de biblioteca
import pandas as pd

In [2]:
# Determinação de constantes
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

dataset = 'ouro_volume_agricola_lavouras'

### Ouro

In [3]:
# Importação de dados de nível prata relativos a lavouras
df_1612_consolidado_uf = pd.read_csv(f'{raiz}/{dataset}_temporarias_uf.csv')

df_1613_consolidado_uf = pd.read_csv(f'{raiz}/{dataset}_permanentes_uf.csv')

In [4]:
# Separação de dados de 1a. safra das culturas de ambos os tipos de lavouras
lavoura_ps = (df_1612_consolidado_uf.filter(['uf', 'ano', 'safra']).loc[df_1612_consolidado_uf.safra == "1a. safra", :].reset_index(drop=True)
              .assign(valor = df_1612_consolidado_uf.loc[df_1612_consolidado_uf.safra == "1a. safra", :].reset_index(drop=True).iloc[:,3] + df_1613_consolidado_uf.iloc[:,2])
              .convert_dtypes()
              )

In [5]:
# Separação de dados de 2a. safra das culturas (encontradas apenas em lavouras temporárias)
lavoura_ss = (df_1612_consolidado_uf.loc[df_1612_consolidado_uf.safra == "2a. safra", :]
              .reset_index(drop=True)
              .convert_dtypes()
              )

In [6]:
# Concatenação das tabelas de 1a. e 2a. safras
lavoura_total = (pd.merge(
                            lavoura_ps.drop(columns='safra'),
                            lavoura_ss.drop(columns='safra'),
                            how='left',
                            on=['uf', 'ano'],
                            suffixes=['_ps', '_ss']
                            )
                        .assign(valor=lambda df: df.apply(
                            lambda x: x.valor_ps + x.valor_ss 
                            if pd.notna(x.valor_ps) and pd.notna(x.valor_ss) else
                            x.valor_ps,
                                axis = 1)) # Soma dos valores de safras
                        .assign(cf=lambda df: df.apply(
                            lambda x: x.valor / x.valor_ps,
                            axis = 1)) # Divisão da soma das safras (descoberta acima) pelo valor da primeira safra
                        .drop(columns=['valor_ps', 'valor_ss'])
                        .convert_dtypes()
 )

# Exportação final dos dados
lavoura_total.to_csv(f'{raiz}/{dataset}_total.csv', index=False)